<a href="https://colab.research.google.com/github/ShinAsakawa/2019seminar_info/blob/master/notebooks/2019si_rnn_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>[Python で 超実習ディープラーニング](https://www.seminar-info.jp/entry/seminars/view/1/4174)</h1>
<h3><strong>実践！ 強化学習・画像認識・自然言語処理・ロボティクス</strong></h3>
</center>

<center>
![](https://www.seminar-info.jp/entry/img/logo_ov.jpg)
</center>

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 15/Mar/2019<br>
 MIT license
</div>

# リカレントニューラルネットワーク

リカレントニューラルネットワーク (Recurrent neural networks:RNN) は系列データ処理のためのニューラルネットワークモデルです。

- 自然言語処理
- 時系列予測
- 自動翻訳

などなど応用範囲が広いです。
時刻 $t$ における入力を $x_t$ とすると，活性化関数を $f$, 中間層の出力を $h$, 出力を $y$ とすれば
リカレントニューラルネットワークの挙動は以下の式のように表現できます。

$$y_{t+1}, h_{t+1} = f(x_{t}, h_{t}),$$

以下の図はグレーブスによる通常のリカレントネットワークの表記(左)とその時間展開(右)になります。
<!--<img src='images/rnn_basic.png', width=600>-->

<center>
<!-- <img src='https://drive.google.com/file/d/1M9fIdT_ezJv8k5STiVNd5k0qExQ4U8fd/'  style="width:600px"> -->
 <img src='http://www.cis.twcu.ac.jp/~asakawa/2019seminar_info/assets/rnn_basic.png' style="width:600px">

</center>

Image by 
[Alex Graves](https://www.cs.toronto.edu/~graves/preprint.pdf)

# エンコーダ=デコーダモデル

カルパシィによる 4 つのニューラルネットワークモデルの分類を以下に示しました。
* **one-to-one** - 通常の多層パーセプトロン。リカレントニューラルネットワークではありません。画像認識などで使われます
* **one-to-many** - 画像から言語を生成する [画像キャプション付けモデル](http://cs.stanford.edu/people/karpathy/deepimagesent/) 
* **Many-to-one** - 文書の極性判断など
* **Many-to-many** 自動翻訳などで使われます。以下のエンコーダ=デコーダモデル参照
* **Many-to-many** ロボット制御などで使われます

<img src="http://www.cis.twcu.ac.jp/~asakawa/2019seminar_info/assets/types.jpeg" style="width:800px">

* [カルパシィのブログ](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*
* [オラーのブログ](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) 

## エンコーダ=デコーダモデル

言語モデルでは，ニューラルネットワークモデルが主流になる前，**長距離依存 long-term dependency** が問題でした。
これを解決するモデルが **LSTM (Long Short-Term Memory) 長短期記憶** になります。

下の図は簡単な翻訳モデルです。

<img src='images/enc-dec.png' style="width:400px">

<!--
#### Teacher forcing
We will also use what is called *teacher forcing*.
This is shown as the gray lines in the figure.
This means that the RNN will implement a sequence of conditional distributions so the $t$th output of the decoder gives $p(y_t|y_1,\dots,y_{t-1} ,x)$. This formulation will make the task easier and faster for the network to learn because it during training always have access to the correct preceding outputs.
A test time where we we don't know the output sequence we have to predict one step at a time. 
There is no guarantee that we will find the mostly likely decoded *sequence*. A technique called [beam search](https://arxiv.org/pdf/1702.01806.pdf) is used in machine translation and related tasks to look for list of candidate decoded sequences.    



#### Alternatives
There are other ways to let the encoder and decoder communicate with each other.
For instance you can give the last state of the Encoder as input to the Decoder at each decode time step, not just the previously predicted word.
Another approache is called **attention**, which lets the Decoder attend to different parts of the encoded input at different timesteps in the decoding process. 
Attention is shown in the next notebook. 
-->


---

# 実習

以下は Keras のサンプルコード
[addtion_rnn.py](https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py) を修正したものです。
3 桁までの足し算を実行するニューラルネットワークになります。
1980年代は **選択的忘却** あるいは **カタストロフィック忘却** と言って，覚えたルールをすぐに忘れるということが問題になっていました。
それを解決するのがリカレントニューラルネットワークモデルで，比較的簡単な実装で体感できます。

一般にリカレントニューラルネットワークは学習に時間を要し，かつ，**黒魔術** と揶揄されてきた特別な技法が必要でした。
これは勾配消失問題，勾配爆発問題と密接に関連します。
勾配チェック，勾配クリップ，勾配正規化，リカレントニューラル層の対角初期化など，本セミナーでは触れません。


In [0]:
# -*- coding: utf-8 -*-
'''
# An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''  
# noqa

# 必要なライブラリを輸入 import します 
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [0]:
class CharacterTable(object):
    """Given a set of characters:
    ワンホットエンコーディングをするクラスの定義
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# パラメータとデータセットの定義
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [0]:
questions = []
expected = []
seen = set()
print('データの生成...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('全問題数:', len(questions))

データの生成...
全問題数: 50000


In [0]:
print('入出力データの準備...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('訓練データ:')
print(x_train.shape)
print(y_train.shape)

print('検証データ:')
print(x_val.shape)
print(y_val.shape)

# LSTM を用いていますが，単純RNN (SimplerNN) や GRU に入れ替えることもできます
RNN = layers.LSTM
HIDDEN_SIZE = 128  # 各層内のニューロン数
BATCH_SIZE = 128   # ミニバッチサイズ
LAYERS = 2         # 中間層数

入出力データの準備...
訓練データ:
(45000, 7, 12)
(45000, 4, 12)
検証データ:
(5000, 7, 12)
(5000, 4, 12)


In [0]:
print('モデル作成...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

モデル作成...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 336,908
Trainable params: 336,908
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 30s 664us/step - loss: 1.9156 - acc: 0.3173 - val_loss: 1.8100 - val_acc: 0.3377
Q 81+175  T 256  ☒ 106 
Q 690+761 T 1451 ☒ 106 
Q 8+11    T 19   ☒ 14  
Q 391+191 T 582  ☒ 106 
Q 47+97   T 144  ☒ 124 
Q 735+953 T 1688 ☒ 106 
Q 5+496   T 501  ☒ 124 
Q 39+622  T 661  ☒ 126 
Q 91+363  T 454  ☒ 106 
Q 9+889   T 898  ☒ 104 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 28s 613us/step - loss: 1.7552 - acc: 0.3467 - val_loss: 1.6589 - val_acc: 0.3679
Q 23+370  T 393  ☒ 144 
Q 7+621   T 628  ☒ 144 
Q 194+927 T 1121 ☒ 1024
Q 71+539  T 610  ☒ 144 
Q 53+95   T 148  ☒ 444 
Q 88+600  T 688  ☒ 856 
Q 141+5   T 146  ☒ 140 
Q 642+723 T 1365 ☒ 1046
Q 865+341 T 1206 ☒